In [ ]:
# ===== 0. Bibliotheken =====
from pathlib import Path
import csv
from typing import Dict

import numpy as np
import torch
from datasets import load_from_disk
from PIL import Image
from tqdm import tqdm
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import albumentations as A

In [ ]:

# ===== 1. Modell & Prozessor =====
MODEL_ID = "merged_model_batchsize_2/"
model = Gemma3ForConditionalGeneration.from_pretrained(
    MODEL_ID,
    device_map={"": 0},
    torch_dtype=torch.bfloat16,
).eval()

processor = AutoProcessor.from_pretrained(MODEL_ID, use_fast=True)

DTYPE = (
    torch.bfloat16
    if torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    else torch.float32
)
device_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
print(f"🖥️ Torch device: {device_name} | Dtype: {DTYPE}")


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
# ===== 2. Daten laden =====
PROJECT_ROOT = Path.cwd().parent
DATA_PATH = PROJECT_ROOT / "data" / "validation"
dataset = load_from_disk(str(DATA_PATH))
print(f"Dataset geladen: {len(dataset)} Samples")

🖥️ Torch device: NVIDIA L4


In [ ]:
# ===== 3. Bild-Augmentierung =====
augment = A.Compose(
    [
        A.Rotate(limit=15, p=0.5),
        A.RandomBrightnessContrast(0.1, 0.1, p=0.5),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
        A.MotionBlur(blur_limit=3, p=0.3),
        A.CoarseDropout(max_holes=5, max_h_size=32, max_w_size=32, p=0.3),
        A.ImageCompression(70, 100, p=0.3),
    ]
)

/tmp/ipykernel_17650/30055526.py:4: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
/tmp/ipykernel_17650/30055526.py:6: UserWarning: Argument(s) 'max_holes, max_h_size, max_w_size' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=5, max_h_size=32, max_w_size=32, p=0.3),
/tmp/ipykernel_17650/30055526.py:7: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=70, quality_upper=100, p=0.3),


In [ ]:
# ===== 4. CSV-Ausgabe vorbereiten =====
OUTPUT_CSV = PROJECT_ROOT / "data" / "llm_answers" / "batchsize_2_imageaug_results.csv"
OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)

FIELDNAMES = ["ID", "question", "correct_answer", "model_output"]

In [ ]:
with OUTPUT_CSV.open("w", newline="", encoding="utf-8") as f_out:
    writer = csv.DictWriter(f_out, fieldnames=FIELDNAMES)
    writer.writeheader()

    # ===== 5. Generierung =====
    for idx in tqdm(range(len(dataset)), desc="Samples"):
        sample = dataset[idx]

        # Bild laden und augmentieren
        image_rgb = sample["image"].convert("RGB")
        aug_np = augment(image=np.array(image_rgb))["image"]
        aug_image = Image.fromarray(aug_np)

        question = sample["question"]
        ground_truth = sample["answer"]
        qid = sample.get("id", idx + 1)

        # Gemma-Chat-Prompt
        messages = [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": (
                            "You are a medical pathology expert. "
                            "Answer strictly based on the visual information in the image. "
                            "Use short precise terms without explanations."
                        ),
                    }
                ],
            },
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": aug_image},
                    {"type": "text", "text": question},
                ],
            },
        ]

        try:
            # Tokenisierung
            inputs = processor.apply_chat_template(
                messages,
                add_generation_prompt=True,
                tokenize=True,
                return_dict=True,
                return_tensors="pt",
            ).to(model.device, dtype=DTYPE)

            prompt_len = inputs["input_ids"].shape[-1]

            # Inferenz
            with torch.inference_mode():
                gen = model.generate(
                    **inputs,
                    max_new_tokens=100,
                    do_sample=False,
                )

            answer = processor.decode(gen[0][prompt_len:], skip_special_tokens=True).strip()

            # CSV-Eintrag
            writer.writerow(
                {
                    "ID": qid,
                    "question": question,
                    "correct_answer": ground_truth,
                    "model_output": answer,
                }
            )

        except Exception as err:
            print(f"Fehler bei Sample {qid}: {err}")
            continue

print(f"\nAlle Ergebnisse gesichert: {OUTPUT_CSV.relative_to(PROJECT_ROOT)}")

  0%|          | 0/6259 [00:00<?, ?it/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  7%|▋         | 429/6259 [02:20<34:51,  2.79it/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
100%|██████████| 6259/6259 [34:41<00:00,  3.01it/s]

✅ All results saved to bildaugmentattion.csv
